# CPU to GPU

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch import nn
import pandas
import math
from torch import optim
import timeit as time
import numpy as np

In [2]:
df = pandas.read_csv("normalized4.csv")
df = df.iloc[:,1:]
# shuffle the dataframe
df = df.sample(frac=1)
df

,entry0,entry1,entry2,entry3,entry4,entry5,entry6,entry7,entry8,entry9,...,feat_DSSP_12,feat_DSSP_13,coord_X,coord_Y,coord_Z,Non_Polar,Polar,Pos_Charge,Neg_Charge,y_Ligand
489837,1,0,1,1,0,0,0,1,1,0,...,0.499671,0.931034,0.484633,0.602180,0.456164,0,1,0,0,0
232736,0,1,0,1,1,0,0,1,1,0,...,0.499452,0.965517,0.594390,0.409853,0.429342,1,0,0,0,0
50410,0,0,0,1,0,0,1,0,0,0,...,0.500986,0.931034,0.605922,0.570167,0.339752,0,1,0,0,0
88821,0,0,1,0,0,0,0,0,0,0,...,0.494631,0.965517,0.467062,0.439949,0.565858,0,1,0,0,0
281495,0,1,1,0,1,0,0,1,0,0,...,0.500329,0.862069,0.635148,0.567420,0.335112,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344890,1,0,0,0,0,0,0,1,0,1,...,0.499671,1.000000,0.309754,0.435781,0.501586,0,0,0,1,0
49203,0,0,0,1,0,0,0,1,1,0,...,0.499890,1.000000,0.403810,0.301221,0.594465,0,0,1,0,0
76786,0,0,0,1,1,1,0,0,0,0,...,0.499890,1.000000,0.612279,0.534077,0.602300,0,1,0,0,0
53648,0,0,0,1,0,0,1,0,1,0,...,0.499890,1.000000,0.621010,0.449630,0.483024,0,0,1,0,0


In [3]:
df2 = df[df['y_Ligand'] == 1]
df2

,entry0,entry1,entry2,entry3,entry4,entry5,entry6,entry7,entry8,entry9,...,feat_DSSP_12,feat_DSSP_13,coord_X,coord_Y,coord_Z,Non_Polar,Polar,Pos_Charge,Neg_Charge,y_Ligand
89925,0,0,1,0,0,0,0,0,1,1,...,0.500986,0.931034,0.499307,0.515637,0.449716,0,1,0,0,1
290109,0,1,1,0,1,1,0,0,0,1,...,0.500329,0.793103,0.413716,0.552154,0.476219,0,1,0,0,1
44928,0,0,0,1,0,0,0,0,0,1,...,0.499452,0.931034,0.470514,0.481652,0.494942,0,0,0,1,1
109641,0,0,1,0,1,0,0,1,0,1,...,0.500329,0.931034,0.443760,0.485188,0.491169,0,0,0,1,1
13705,0,0,0,0,0,1,1,0,0,0,...,0.500548,0.793103,0.481286,0.556786,0.468341,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63647,0,0,0,1,0,1,1,1,0,0,...,0.529915,0.965517,0.483924,0.486784,0.447475,0,0,1,0,1
455928,1,0,1,0,0,1,1,0,1,1,...,0.500548,0.896552,0.437018,0.518380,0.505967,0,0,1,0,1
43325,0,0,0,0,1,1,1,1,0,1,...,0.500548,0.965517,0.550266,0.533403,0.422519,0,0,1,0,1
286282,0,1,1,0,1,0,1,1,0,1,...,0.501205,0.724138,0.445396,0.601822,0.547670,0,1,0,0,1


In [4]:
for i in range(40):
    df = df.append(df2)
df

C:\Users\Wyett\AppData\Local\Temp\ipykernel_17240\2102259913.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
C:\Users\Wyett\AppData\Local\Temp\ipykernel_17240\2102259913.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
C:\Users\Wyett\AppData\Local\Temp\ipykernel_17240\2102259913.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
C:\Users\Wyett\AppData\Local\Temp\ipykernel_17240\2102259913.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
C:\Users\Wyett\AppData\Local\Temp\ipykernel_17240\2102259913.py:2: FutureWarning: The frame.append method is dep

,entry0,entry1,entry2,entry3,entry4,entry5,entry6,entry7,entry8,entry9,...,feat_DSSP_12,feat_DSSP_13,coord_X,coord_Y,coord_Z,Non_Polar,Polar,Pos_Charge,Neg_Charge,y_Ligand
489837,1,0,1,1,0,0,0,1,1,0,...,0.499671,0.931034,0.484633,0.602180,0.456164,0,1,0,0,0
232736,0,1,0,1,1,0,0,1,1,0,...,0.499452,0.965517,0.594390,0.409853,0.429342,1,0,0,0,0
50410,0,0,0,1,0,0,1,0,0,0,...,0.500986,0.931034,0.605922,0.570167,0.339752,0,1,0,0,0
88821,0,0,1,0,0,0,0,0,0,0,...,0.494631,0.965517,0.467062,0.439949,0.565858,0,1,0,0,0
281495,0,1,1,0,1,0,0,1,0,0,...,0.500329,0.862069,0.635148,0.567420,0.335112,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63647,0,0,0,1,0,1,1,1,0,0,...,0.529915,0.965517,0.483924,0.486784,0.447475,0,0,1,0,1
455928,1,0,1,0,0,1,1,0,1,1,...,0.500548,0.896552,0.437018,0.518380,0.505967,0,0,1,0,1
43325,0,0,0,0,1,1,1,1,0,1,...,0.500548,0.965517,0.550266,0.533403,0.422519,0,0,1,0,1
286282,0,1,1,0,1,0,1,1,0,1,...,0.501205,0.724138,0.445396,0.601822,0.547670,0,1,0,0,1


In [ ]:
df.to_csv("duplicated.csv")

In [5]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [27]:
class MyDataset(Dataset):
 
  def __init__(self,df): 
    x=df.iloc[:,:-1].values
    y=df.iloc[:,-1:].values
    self.x_train=torch.tensor(x,dtype=torch.float32)
    self.y_train=torch.tensor(y,dtype=torch.long)
    self.y_train = self.y_train.view(-1)
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [28]:
indexCutOff = int(len(df) *0.9)

# Download FMNIST training dataset and load training data
trainset = MyDataset(df[:indexCutOff])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=30000, shuffle=True)

# Download FMNIST test dataset and load test data
testset = MyDataset(df[indexCutOff:])
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [29]:
class Cyclica(nn.Module):
  def __init__(self, column_size):
    super().__init__()
    pw = math.ceil(math.log(column_size, 2))
    self.fc1 = nn.Linear(column_size, int(math.pow(2, pw - 1)))
    self.fc2 = nn.Linear(int(math.pow(2, pw - 1)),int(math.pow(2, pw - 2)))
    self.fc3 = nn.Linear(int(math.pow(2, pw - 2)), 1)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = F.log_softmax(x, dim=1)
    
    return x 

model = Cyclica(len(df.columns) - 1)

In [31]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

num_epochs = 1024

for i in range(num_epochs):
    cum_loss = 0

    start = time.default_timer()
    
    for images, labels in trainloader:
        optimizer.zero_grad()    

        output = model(images)
        print(output.shape)
        print(labels.shape)
        loss = criterion(output, labels)
        #print("output: " + str(output) + ", labels: " + str(labels))
        loss.backward()
        optimizer.step()
        
        
        cum_loss += loss.item()
    stop = time.default_timer()
    print(f"run {i + 1}, Training loss: {cum_loss/len(trainloader)}, run on the CPU with time {stop - start}")

torch.Size([30000, 1])
torch.Size([30000])


ValueError: Using a target size (torch.Size([30000])) that is different to the input size (torch.Size([30000, 1])) is deprecated. Please ensure they have the same size.